In [1]:
using LinearAlgebra, LightGraphs, SparseArrays, SimpleWeightedGraphs, Distances, ProgressMeter
using PyCall, Conda, OrderedCollections
nx = pyimport("networkx")

PyObject <module 'networkx' from '/Users/rishisonthalia/.julia/conda/3/lib/python3.8/site-packages/networkx/__init__.py'>

In [10]:
"""
  Input:
    G is a networkx weighted graph, weight is given by the term weight. The graph has n nodes labelled 0,...,n-1 and m edges
  Output:
    A - m x m matrix. Has a 1 in Aij is edge i and edge j are transverse.
    edge
"""
function create_views(G)
    # Get shortest path metric on the weighted graph
    n = nv(G)
    D = LightGraphs.floyd_warshall_shortest_paths(G).dists

    # Create edge to edge_index dictionary
    edge_to_index = Dict()
    L = enumerate(edges(G))
    m = length(L)
    for idx in L
        i,e = idx
        edge_to_index[e] = i
    end
    
    A = zeros(m,m)
    @showprogress for (i,ei) in L
        for (j,ej) in L
              if i != j
                A[i,j] = correct_answer(D,ei,ej)
            end
        end
    end
    # print((A-A.T).abs().sum())

    return A, edge_to_index, collect(L), D
end

"""
  Input :
    D - n x n distance matrix
    ei - edge i
    ej - edge j
  Outut :
    0 if ei and ej are independent
    1 if ei and ej are not independent
  Decription :
    Here we see that ei = (a,b) and ej = (u,v) are independent
    if there exists au such that au is closer to a than to b and
    to u than to v.
    Similarly, for av, bu, bv.
"""
function correct_answer(D,ei,ej)
    a = ei.src
    b = ei.dst
    u = ej.src
    v = ej.dst

    au = false
    av = false
    bu = false
    bv = false

    n = size(D)[0+1]
    for i = 1:n
        if D[i,a] < D[i,b]
            if D[i,u] < D[i,v]
                au = true
            elseif D[i,v] < D[i,u]
                av = true
            else
                au = true
                av = true
            end
        elseif D[i,b] < D[i,a]
            if D[i,u] < D[i,v]
                bu = true
            elseif D[i,v] < D[i,u]
                bv = true
            else
                bu = true
                bv = true
            end
        else
            if D[i,u] < D[i,v]
                au = true
                bu = true
            elseif D[i,v] < D[i,u]
                av = true
                bv = true
            else
                au = true
                av = true
                bu = true
                bv = true
            end
        end
    end


    if au && av && bu && bv
      return 0
    else
      return 1
    end
end

"""
  Inputs :
    G is the unweighted graph
    k is the number of edges to add to the graph. Default : None
  Outputs :
    Rewired graph
  Description :
    The process is done as follows
      1) First figure out which edges are independent.
      2) Next count how often un-adjacent node pairs appear in
         non-independent pairs multiplied by distance in graph
      3) Connect the top k node pairs that appear. If k is none
         we connect all node pairs.
"""
function rewire_hhs(G,k,A,edge_to_index,L,D,dist=true)
    n = size(D)[0+1]
    m = size(A)[0+1]
    counts = OrderedDict()
    for i = 1:m
        for j = 1:i-1
            if A[i,j] > 0.1
                i,ei = L[i]
                j,ej = L[j]
                
                keys_nodes = [(ei.src,ej.src),(ei.src,ej.dst),(ei.dst,ej.src),(ei.dst,ej.dst)]
                
                for key in keys_nodes
                    if !has_edge(G,key[0+1],key[1+1])
                        if dist
                            if key in keys(counts)
                                counts[key] += D[key[0+1], key[1+1]]
                            else
                                counts[key] = D[key[0+1],key[1+1]]
                            end
                        else
                            if key in keys(counts)
                                counts[key] += 1
                            else
                                counts[key] = 1
                            end
                        end
                    end
                end
            end
        end
    end
    
    sorted_counts = sort(counts, byvalue = true, rev=true)
    
    # print(sorted_counts)
    if k > length(sorted_counts)
        k = length(sorted_counts)
    end

    key_edge = collect(keys(sorted_counts))
    for i = 1:k
        u = key_edge[i][1]
        v = key_edge[i][2]
        add_edge!(G,u,v)
    end
    return G
end

rewire_hhs

In [3]:
G = SimpleGraph(15)
for i = 1:5
    for j = 1:i-1
        add_edge!(G,i,j)
        add_edge!(G,i+10,j+10)
    end
end

for i = 5:10
    add_edge!(G,i,i+1)
end

In [4]:
using Plots, GraphRecipes
plotly()
graphplot(G, curves=false, names=1:nv(G), fontsize = 20)

[ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.


<!DOCTYPE html>
 
 
 Plots.jl

In [5]:
A, edge_to_index, L, D = create_views(G)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], Dict{Any,Any}(Edge 12 => 15 => 23,Edge 3 => 4 => 8,Edge 8 => 9 => 14,Edge 12 => 13 => 21,Edge 3 => 5 => 9,Edge 4 => 5 => 10,Edge 1 => 5 => 4,Edge 1 => 2 => 1,Edge 1 => 4 => 3,Edge 11 => 15 => 20…), Tuple{Int64,LightGraphs.SimpleGraphs.SimpleEdge{Int64}}[(1, Edge 1 => 2), (2, Edge 1 => 3), (3, Edge 1 => 4), (4, Edge 1 => 5), (5, Edge 2 => 3), (6, Edge 2 => 4), (7, Edge 2 => 5), (8, Edge 3 => 4), (9, Edge 3 => 5), (10, Edge 4 => 5)  …  (17, Edge 11 => 12), (18, Edge 11 => 13), (19, Edge 11 => 14), (20, Edge 11 => 15), (21, Edge 12 => 13), (22, Edge 12 => 14), (23, Edge 12 => 15), (24, Edge 13 => 14), (25, Edge 13 => 15), (26, Edge 14 => 15)], [0 1 … 8 8; 1 0 … 8 8; … ; 8 8 … 0 1; 8 8 … 1 0])

In [6]:
G2 = rewire_hhs(G,1,A,edge_to_index,L,D)

{15, 27} undirected simple Int64 graph

In [3]:
py"""
import pickle
 
def load_pickle(fpath):
    with open(fpath, "rb") as f:
        data = pickle.load(f)
    return data
"""

load_pickle = py"load_pickle"

PyObject <function load_pickle at 0x7fa26065dca0>

In [7]:
graphs = [load_pickle("./cora.pkl")];
print(length(graphs))
graphs

1

1-element Array{Array{Int64,2},1}:
 [0 0 … 2707 2707; 633 1862 … 1473 2706]

In [8]:
N = length(graphs)
Gs = []
for i = 1:N
    n = maximum(graphs[i])+1
    G = SimpleGraph(n)
    m = size(graphs[i])[2]
    for j = 1:m
        u = graphs[i][1,j]+1
        v = graphs[i][2,j]+1
        add_edge!(G,u,v)
    end
    push!(Gs,G)
end

In [13]:
# rewired_Gs = []
# A, edge_to_index, L, D = create_views(Gs[1])
@showprogress for i = 1:1000
    rewired_G = rewire_hhs(copy(Gs[1]),i,A,edge_to_index,L,D)
    rewired_D = LightGraphs.floyd_warshall_shortest_paths(rewired_G).dists
    push!(rewired_Gs,(rewired_G,rewired_D))
end

Progress: 100%|█████████████████████████████████████████| Time: 7:52:55


In [14]:
As = []
@showprogress for i = 1:1000
    push!(As, LightGraphs.adjacency_matrix(rewired_Gs[i][1]), rewired_Gs[i][2])
end

Progress: 100%|█████████████████████████████████████████| Time: 0:00:03


In [15]:
py"""
import pickle
 
def save_pickle(fpath,A):
    with open(fpath, "wb") as f:
        pickle.dump(A,f,protocol=pickle.HIGHEST_PROTOCOL)
"""
save_pickle = py"save_pickle"

PyObject <function save_pickle at 0x7fa260f02d30>

In [ ]:
save_pickle("cora-hhs-rewired-1000.pkl", As)